In [0]:
from pyspark.sql import SparkSession

spark = SparkSession \
    .builder \
    .master("local[*]") \
    .appName("FinalCoursework") \
    .config("spark.driver.memory", "4g")\
    .getOrCreate()

sc = spark.sparkContext

Read the File

In [0]:
dir_path="/mnt/data/project-data/2020tweets"

df_trump = spark.read.option("multiline", True).csv(dir_path+"/hashtag_donaldtrump.csv", header=True, inferSchema=True)
df_biden = spark.read.option("multiline", True).csv(dir_path+"/hashtag_joebiden.csv", header=True, inferSchema=True)


In [0]:
def transer_data_type(data):
  data = data.withColumn("likes", data["likes"].cast("int"))
  data = data.withColumn("retweet_count", data["retweet_count"].cast("int"))
  data = data.withColumn("user_followers_count", data["user_followers_count"].cast("int"))
  data = data.withColumn("lat", data["lat"].cast("float"))
  data = data.withColumn("long", data["long"].cast("float"))
  return data

df_trump = transer_data_type(df_trump)
df_biden = transer_data_type(df_biden)

In [0]:
#Dropping columns
remove_columns = ['created_at', 'tweet_id', 'user_id', 'user_screen_name', 'user_join_date', 'collected_at', 'user_name', 'user_description', 'user_location', 'lat', 'long', 'city', 'country', 'continent', 'state', 'state_code']
df_trump = df_trump.drop(*remove_columns)
df_biden = df_biden.drop(*remove_columns)

#Dropping rows with na values
df_trump = df_trump.na.drop()
df_biden = df_biden.na.drop()

Compared the number between biden and trump

In [0]:
#import matplotlib.pyplot as plt

# Create a bar plot
#plt.bar(['Trump', 'Biden'], [df_trump.count(), df_biden.count()])
#plt.xlabel('Candidate')
#plt.ylabel('Number of Tweets')
#plt.title('Number of Tweets for Trump and Biden')
#plt.show()

In [0]:
from pyspark.sql.functions import lit

#Adding a presidents column where 0 represents trump and 1 represents biden. This will turn it into a classification problem
df_trump = df_trump.withColumn('President', lit(0))
df_biden = df_biden.withColumn('President', lit(1))

#Merging the 2 datasets
df = df_trump.union(df_biden)

In [0]:
df.show(5)

+--------------------+-----+-------------+------------------+--------------------+---------+
|               tweet|likes|retweet_count|            source|user_followers_count|President|
+--------------------+-----+-------------+------------------+--------------------+---------+
|#Elecciones2020 |...|    0|            0|         TweetDeck|                1860|        0|
|Usa 2020, Trump c...|   26|            9|  Social Mediaset |             1067661|        0|
|#Trump: As a stud...|    2|            1|   Twitter Web App|                1185|        0|
|2 hours since las...|    0|            0|     Trumpytweeter|                  32|        0|
|You get a tie! An...|    4|            3|Twitter for iPhone|                5393|        0|
+--------------------+-----+-------------+------------------+--------------------+---------+
only showing top 5 rows



Caculate HashTags

In [0]:
from pyspark.sql.functions import udf
from pyspark.sql.types import ArrayType, StringType
from collections import Counter
import re

#Extracts all the hastags from the tweet and stores them in a list
@udf(returnType=ArrayType(StringType()))
def getHashtagList(tweet):
  tweet = tweet.lower()
  tweet = re.sub(r'[^\w\s#]', '', tweet)
  hashtag_list = []

  for word in tweet.split():
    if word[0] == '#':
      hashtag_list.append(word)
  
  #Creates a dictionary where the hashtags are the key and their frequency in the list is the value
  count_hashtags = Counter(hashtag_list)

  #Remove duplicate hashtags
  unique_hashtags = [hashtag for hashtag, frequency in count_hashtags.items() if frequency == 1]

  return hashtag_list

#Creates a column called hashtags which stores a hastag list for all the tweets
df = df.withColumn('hashtags', getHashtagList(df['tweet']))

In [0]:
df.show(5)

+--------------------+-----+-------------+------------------+--------------------+---------+--------------------+
|               tweet|likes|retweet_count|            source|user_followers_count|President|            hashtags|
+--------------------+-----+-------------+------------------+--------------------+---------+--------------------+
|#Elecciones2020 |...|    0|            0|         TweetDeck|                1860|        0|[#elecciones2020,...|
|Usa 2020, Trump c...|   26|            9|  Social Mediaset |             1067661|        0|      [#donaldtrump]|
|#Trump: As a stud...|    2|            1|   Twitter Web App|                1185|        0|            [#trump]|
|2 hours since las...|    0|            0|     Trumpytweeter|                  32|        0|            [#trump]|
|You get a tie! An...|    4|            3|Twitter for iPhone|                5393|        0|     [#trump, #iowa]|
+--------------------+-----+-------------+------------------+--------------------+------

In [0]:
from pyspark.sql import functions as F

#Splits the hashtag list and displays each hashtag in its own row in column hashtag
df_exploded = df.withColumn('hashtag', F.explode(df['hashtags']))

#Creates a new column called count which displays how many rows the hashtag appears in
#The counts will be used in TF-IDF calculation
df_count = df_exploded.groupBy("hashtag").count()

#Renaming count to hashtag_count
df_count = df_count.withColumnRenamed('count', 'hashtag_count')

#Joining df_count with df_exploded on hashtag
df_exploded = df_exploded.join(df_count, on="hashtag", how="left")

# Group by original columns and get the list of counts column
df = df_exploded.groupBy(df.columns).agg(F.collect_list("hashtag_count").alias("hashtag_counts"))

In [0]:
df.show(5)

+--------------------+-----+-------------+-------------------+--------------------+---------+--------------------+--------------------+
|               tweet|likes|retweet_count|             source|user_followers_count|President|            hashtags|      hashtag_counts|
+--------------------+-----+-------------+-------------------+--------------------+---------+--------------------+--------------------+
| He’s nothing mor...|    0|            0|    Twitter Web App|                  71|        0|            [#trump]|            [968674]|
| it shows that #t...|    1|            0|Twitter for Android|                  11|        0|            [#trump]|            [968674]|
| media-sourced po...|    0|            0|Twitter for Android|                 657|        0|            [#biden]|            [631862]|
| mientras hace ca...|    0|            0| Twitter for iPhone|                  14|        1|[#electionday, #d...|[631862, 78352, 6...|
| so Please get a ...|    0|            0|Twitte

In [0]:
#Total number of rows
#This value is used to calculate TF-IDF
df_count = df.count()
print(df_count)

1643761


In [0]:
from pyspark.sql.types import FloatType
import math

#Creates the TF_IDF list column
@udf(returnType=FloatType())
def getAvg_TF_IDF(hashtags, hashtag_counts, df_count):
  avg_TF_IDF = 0
  count = 0

  #If there are no hashtags the tweet will be given a value of 0
  if len(hashtags) == 0:
    return 0.0

  for hashtag in hashtags:
    #Calculating TF
    #Since all the hastags are unique 1 is being divided by the total number of hastags
    TF = 1 / len(hashtags)
    #nDocs is the number of rows/tweets the hastag is used
    nDocs = hashtag_counts[count]
    #Calculating IDF
    #df_count is the total number of rows/tweets
    IDF = math.log(df_count / nDocs)
    TF_IDF = TF * IDF
    avg_TF_IDF += TF_IDF
    count += 1

  avg_TF_IDF /= count
  return avg_TF_IDF

# Apply the UDF to your DataFrame
df = df.withColumn('TF_IDF', getAvg_TF_IDF(df['hashtags'], df['hashtag_counts'], lit(df_count)))

In [0]:
df.show(5)

+--------------------+-----+-------------+-------------------+--------------------+---------+--------------------+--------------------+---------+
|               tweet|likes|retweet_count|             source|user_followers_count|President|            hashtags|      hashtag_counts|   TF_IDF|
+--------------------+-----+-------------+-------------------+--------------------+---------+--------------------+--------------------+---------+
| He’s nothing mor...|    0|            0|    Twitter Web App|                  71|        0|            [#trump]|            [968674]|0.5288141|
| it shows that #t...|    1|            0|Twitter for Android|                  11|        0|            [#trump]|            [968674]|0.5288141|
| media-sourced po...|    0|            0|Twitter for Android|                 657|        0|            [#biden]|            [631862]|0.9560712|
| mientras hace ca...|    0|            0| Twitter for iPhone|                  14|        1|[#electionday, #d...|[631862, 7

We remove stop_words,special punction and url from the text

In [0]:
import re,string
# This code defines a function named "tokenize" that tokenizes the input text.
# It removes special characters, converts the text to lowercase, and removes stopwords and punctuation.
# The resulting tokens are added to a list and returned.

# This code defines a list of stopwords to be removed from the text.
stopwords = [u'rt', u're', u'i', u'me', u'my', u'myself', u'we', u'our', u'ours', u'ourselves', u'you', u'your',
             u'yours', u'yourself', u'yourselves', u'he', u'him', u'his', u'himself', u'she', u'her', u'hers',
             u'herself', u'it', u'its', u'itself', u'they', u'them', u'their', u'theirs', u'themselves', u'what',
             u'which', u'who', u'whom', u'this', u'that', u'these', u'those', u'am', u'is', u'are', u'was', u'were',
             u'be', u'been', u'being', u'have', u'has', u'had', u'having', u'do', u'does', u'did', u'doing', u'a',
             u'an', u'the', u'and', u'but', u'if', u'or', u'because', u'as', u'until', u'while', u'of', u'at', u'by',
             u'for', u'with', u'about', u'against', u'between', u'into', u'through', u'during', u'before', u'after',
             u'above', u'below', u'to', u'from', u'up', u'down', u'in', u'out', u'on', u'off', u'over', u'under',
             u'again', u'further', u'then', u'once', u'here', u'there', u'when', u'where', u'why', u'how', u'all',
             u'any', u'both', u'each', u'few', u'more', u'most', u'other', u'some', u'such', u'no', u'nor', u'not',
             u'only', u'own', u'same', u'so', u'than', u'too', u'very', u's', u't', u'can', u'will', u'just', u'don',
             u'should', u'now']
def tokenize(text):
    # Remove URLs
    text = re.sub('http[s]?://(?:[a-zA-Z]|[0-9]|[$-_@.&+]|[!*(),]|(?:%[0-9a-fA-F][0-9a-fA-F]))+', '', text)
    
    # Remove special characters
    text = re.sub('[%s]' % re.escape(string.punctuation), ' ', text)
    
    # Convert text to lowercase
    text = text.lower()
    
    # Split text into tokens and filter out stopwords, single characters, and '``'
    tokens = [word for word in text.split() if word not in stopwords and len(word) > 1 and word != '``']
    
    return tokens

caculate polarity with sentiment analysis

In [0]:
import nltk
nltk.download('vader_lexicon')
from nltk.sentiment import SentimentIntensityAnalyzer
sia = SentimentIntensityAnalyzer()
sc.broadcast(sia)

def calculate_sentiment(text):
    #sia = SentimentIntensityAnalyzer()
    if isinstance(text, list):
        text = ' '.join(text)
    sentiment_scores = sia.polarity_scores(text)['compound']

    '''if(sentiment_scores>0):
      sentiment_scores=1
    elif(sentiment_scores<0):
      sentiment_scores=-1
    else:
      sentiment_scores=0'''
    return sentiment_scores

[nltk_data] Downloading package vader_lexicon to /home/spark-
[nltk_data]     ee539b33-28e5-4532-9667-19/nltk_data...


In [0]:
from pyspark.sql.types import *

# Define UDF to tokenize the tweet attribute
@udf(returnType=FloatType())
def tokenize_tweet(text):
    # Write code here to tokenize the tweet using the desired method/function
    tokens = tokenize(text)
    sent=calculate_sentiment(tokens)
    return sent

# Add a new column 'Tokens' to the DataFrame 'df' by calling the 'tokenize_tweet' UDF on the 'tweet' column
df=df.withColumn('polarity', tokenize_tweet(df['tweet']))


In [0]:
df.show(5)

+--------------------+-----+-------------+-------------------+--------------------+---------+--------------------+--------------------+---------+--------+
|               tweet|likes|retweet_count|             source|user_followers_count|President|            hashtags|      hashtag_counts|   TF_IDF|polarity|
+--------------------+-----+-------------+-------------------+--------------------+---------+--------------------+--------------------+---------+--------+
| He’s nothing mor...|    0|            0|    Twitter Web App|                  71|        0|            [#trump]|            [968674]|0.5288141|  0.0772|
| it shows that #t...|    1|            0|Twitter for Android|                  11|        0|            [#trump]|            [968674]|0.5288141|  -0.128|
| media-sourced po...|    0|            0|Twitter for Android|                 657|        0|            [#biden]|            [631862]|0.9560712|  0.7579|
| mientras hace ca...|    0|            0| Twitter for iPhone|        

Labeled the cloassfication
if polarity>0 sent=1
if polarity=0 sent=0
if polarity<0 sent=-1

In [0]:
'''from pyspark.sql.functions import when, col

df = df.withColumn('sent', when(col('polarity') > 0, 1)
                        .when(col('polarity') <= 0, -1)
                        .otherwise(0))'''

In [0]:
df.show(5)

+--------------------+-----+-------------+-------------------+--------------------+---------+--------------------+--------------------+---------+--------+----+
|               tweet|likes|retweet_count|             source|user_followers_count|President|            hashtags|      hashtag_counts|   TF_IDF|polarity|sent|
+--------------------+-----+-------------+-------------------+--------------------+---------+--------------------+--------------------+---------+--------+----+
| He’s nothing mor...|    0|            0|    Twitter Web App|                  71|        0|            [#trump]|            [968674]|0.5288141|  0.0772|   1|
| it shows that #t...|    1|            0|Twitter for Android|                  11|        0|            [#trump]|            [968674]|0.5288141|  -0.128|  -1|
| media-sourced po...|    0|            0|Twitter for Android|                 657|        0|            [#biden]|            [631862]|0.9560712|  0.7579|   1|
| mientras hace ca...|    0|            

To use word2Vec,we download the pre-trained word2Vec model and broatcast them to the worker computer


In [0]:
from pyspark import SQLContext
sqlContext = SQLContext(sc)
lookup = sqlContext.read.parquet("/FileStore/word2vecM_simple/data").alias("lookup")
lookup.printSchema()
lookup_bd = sc.broadcast(lookup.rdd.collectAsMap())

/databricks/spark/python/pyspark/sql/context.py:117: FutureWarning: Deprecated in 3.0.0. Use SparkSession.builder.getOrCreate() instead.
  warnings.warn(


root
 |-- word: string (nullable = true)
 |-- vector: array (nullable = true)
 |    |-- element: float (containsNull = true)



In [0]:
# Change the return type of the function from ndarray to list
def doc2vec(document):
    doc_vec = [0.0] * 100
    tot_words = 0
    
    for word in document:
        try:
            vec = lookup_bd.value.get(word)
            if vec is not None:
                doc_vec = [a + b + 1 for a, b in zip(doc_vec, vec)]
                tot_words += 1
        except:
            continue
    
    if tot_words > 0:
        doc_vec = [a / float(tot_words) for a in doc_vec]
    
    return doc_vec

In [0]:
from pyspark.ml.linalg import DenseVector, VectorUDT
from pyspark.sql.functions import udf

# Define UDF to calculate the vector representation of the document by summing up the feature values of each word in the document
@udf(returnType=VectorUDT())
def doc_to_vec(tokens):
    token = tokenize(tokens)
    # Write code here to calculate the vector representation of the document using the desired method/function
    vector = doc2vec(token)
    return DenseVector(vector)
  
# Add a new column 'DocVec' to the DataFrame 'df' by calling the 'doc_to_vec' UDF on the 'Tokens' column
df = df.withColumn('DocVec', doc_to_vec(df['tweet']))

In [0]:
df.show(5)

+--------------------+-----+-------------+-------------------+--------------------+---------+--------------------+--------------------+---------+--------+--------------------+
|               tweet|likes|retweet_count|             source|user_followers_count|President|            hashtags|      hashtag_counts|   TF_IDF|polarity|              DocVec|
+--------------------+-----+-------------+-------------------+--------------------+---------+--------------------+--------------------+---------+--------+--------------------+
| He’s nothing mor...|    0|            0|    Twitter Web App|                  71|        0|            [#trump]|            [968674]|0.5288141|  0.0772|[1.04750266564743...|
| it shows that #t...|    1|            0|Twitter for Android|                  11|        0|            [#trump]|            [968674]|0.5288141|  -0.128|[0.94366044364869...|
| media-sourced po...|    0|            0|Twitter for Android|                 657|        0|            [#biden]|      

Build Train_Data and Split Test and train Data

In [0]:
seed = 654321

train_df, test_df = df.randomSplit([0.7, 0.3], seed=seed)

In [0]:
train_df.cache()

DataFrame[tweet: string, likes: int, retweet_count: int, source: string, user_followers_count: int, President: int, hashtags: array<string>, hashtag_counts: array<bigint>, TF_IDF: float, polarity: float, DocVec: vector]

In [0]:
test_df.cache()

DataFrame[tweet: string, likes: int, retweet_count: int, source: string, user_followers_count: int, President: int, hashtags: array<string>, hashtag_counts: array<bigint>, TF_IDF: float, polarity: float, DocVec: vector]

In [0]:
'''test_size = test_df.count()
print(test_size)'''

493285


In [0]:
from pyspark.ml.feature import VectorAssembler,StringIndexer
feature_columns = ['likes', 'retweet_count', 'user_followers_count','polarity','TF_IDF']
#feature_columns=df.columns
print(feature_columns)
assembler = VectorAssembler(inputCols=feature_columns, outputCol="features")

['likes', 'retweet_count', 'user_followers_count', 'polarity', 'TF_IDF']


In [0]:
from pyspark.ml import Pipeline
from pyspark.ml.classification import RandomForestClassifier
from pyspark.ml.evaluation import MulticlassClassificationEvaluator

# Create the RandomForestClassifier
rf = RandomForestClassifier(featuresCol="features", labelCol="President")

# Create the pipeline with the VectorAssembler and RandomForestClassifier
pipeline = Pipeline(stages=[assembler, rf])

# Fit the pipeline on the training data
model = pipeline.fit(train_df)

# Make predictions on the test data
predictions = model.transform(test_df)

# Evaluate the accuracy of the predictions
evaluator = MulticlassClassificationEvaluator(labelCol="President", metricName="accuracy")
accuracy = evaluator.evaluate(predictions)

accuracy

0.614883890651449